In [ ]:
import msmrd2
import msmrd2.visualization as msmrdvis
import numpy as np

In [ ]:
# Define arbitrarily large list of rod-like particles
numparticles = 400
boxsize = 70
D = 0.1
Drot = 0.1
bodytype = 'rod'
pyPartlist = [] 
for i in range(numparticles):
    overlap = True
    while overlap:
        position = np.array([boxsize*np.random.rand()-0.5*boxsize,boxsize*np.random.rand()-0.5*boxsize,0])
        overlap = False
        for j in range(len(pyPartlist)):
            if np.linalg.norm(position - pyPartlist[j].position) < 2.5:
                overlap = True
                break
    orientation = np.array([np.random.rand(),np.random.rand(),np.random.rand(),np.random.rand()])
    orientation = orientation/np.linalg.norm(orientation)
    #orientation = np.array([np.cos(np.pi/4.0), np.sin(np.pi/4), 0.0, 0.0]) #Rotate 90 degrees in x axis
    part = msmrd2.particle(D, Drot, bodytype, position, orientation)
    pyPartlist.append(part)

In [ ]:
# Create list of particles that can be read from msmrd
# Note the particles in this list will be independent from the python list.
partlist = msmrd2.integrators.particleList(pyPartlist)

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.005
seed = 0 # Seed = -1 used random device as seed
rotation = True
intg = msmrd2.integrators.odLangevin(dt, seed, rotation) 
intg.setKbT(0.1)

In [ ]:
# Define boundary
boxBoundary = msmrd2.box(boxsize,boxsize,0.5,'periodic')
intg.setBoundary(boxBoundary)

In [ ]:
# Define Gay Berne potential
gayBerne = msmrd2.potentials.gayBerne(3.0,4.0,1000.0,1.0)
intg.setPairRodPotential(gayBerne)

In [ ]:
# Integrate particle list and print only positions 
timesteps = 10000
datafile  = open('../data/vmd/gayBerne.xyz', 'w')
for i in range(timesteps):
    datafile.write(str(2*len(partlist)) + '\n')
    datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        v0 = part.position
        v1 = v0 + 1.7*part.orientvector
        v2 = v0 - 1.7*part.orientvector
        datafile.write('type_0' + ' ' + ' '.join(map(str, v1)) + '\n')
        datafile.write('type_0' + ' ' + ' '.join(map(str, v2)) + '\n')
    intg.integrate(partlist)
datafile.close()
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_gayBerne(numparticles = numparticles)

In [ ]:
# Integrate particle list and print only positions of first and last particle in list
timesteps = 100
print('{:<10s}{:<15s}{:<40s}{:<40s}'.format("Iteration", "Time", "Position 1", "Position2"))
for i in range(timesteps):
    print('{:<10d}{:<15f}{:<40s}{:<50s}'.format(i, intg.clock, str(partlist[0].position), str(partlist[numparticles-1].position)))
    intg.integrate(partlist)